In [32]:
import pandas as pd
from fancyimpute import KNN
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# 갑자기 든 생각은 날짜가 전부 다르다면, 비슷한 날씨와 오존량, 가시성, 시간대의 특성을
# 내가 임의로 나누어서, 거기서 가장 비슷한 근처5개 값으로 채우고 모델을 학습시킨다면 더욱 점수가 잘 나오지 않을까? 생각함

In [33]:
train = pd.read_csv('./data/train.csv') 
test = pd.read_csv('data/test.csv')

In [34]:
train

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0
...,...,...,...,...,...,...,...,...,...,...,...
1454,2174,4,16.8,0.0,1.6,53.0,2000.0,0.031,37.0,27.0,21.0
1455,2175,3,10.8,0.0,3.8,45.0,2000.0,0.039,34.0,19.0,20.0
1456,2176,5,18.3,0.0,1.9,54.0,2000.0,0.009,30.0,21.0,22.0
1457,2178,21,20.7,0.0,3.7,37.0,1395.0,0.082,71.0,36.0,216.0


In [35]:
train.describe()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
count,1459.000000,1459.000000,1457.000000,1457.000000,1450.000000,1457.000000,1457.000000,1383.000000,1369.000000,1342.000000,1459.000000
mean,1105.914325,11.493489,16.717433,0.031572,2.479034,52.231297,1405.216884,0.039149,57.168736,30.327124,108.563400
std,631.338681,6.922790,5.239150,0.174917,1.378265,20.370387,583.131708,0.019509,31.771019,14.713252,82.631733
min,3.000000,0.000000,3.100000,0.000000,0.000000,7.000000,78.000000,0.003000,9.000000,8.000000,1.000000
25%,555.500000,5.500000,12.800000,0.000000,1.400000,36.000000,879.000000,0.025500,36.000000,20.000000,37.000000
50%,1115.000000,11.000000,16.600000,0.000000,2.300000,51.000000,1577.000000,0.039000,51.000000,26.000000,96.000000
75%,1651.000000,17.500000,20.100000,0.000000,3.400000,69.000000,1994.000000,0.052000,69.000000,37.000000,150.000000
max,2179.000000,23.000000,30.000000,1.000000,8.000000,99.000000,2000.000000,0.125000,269.000000,90.000000,431.000000


In [36]:
test

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
0,0,7,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0
1,1,17,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0
2,2,13,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0
3,4,6,22.5,0.0,2.5,60.0,1185.0,0.027,52.0,38.0
4,5,22,14.6,1.0,3.4,93.0,218.0,0.041,18.0,15.0
...,...,...,...,...,...,...,...,...,...,...
710,2148,1,24.6,0.0,2.4,60.0,1745.0,NaN,NaN,NaN
711,2149,1,18.1,0.0,1.0,55.0,2000.0,NaN,NaN,NaN
712,2165,9,23.3,0.0,2.3,66.0,1789.0,0.020,17.0,15.0
713,2166,16,27.0,0.0,1.6,46.0,1956.0,0.032,40.0,26.0


In [37]:
test.describe()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
count,715.000000,715.000000,714.000000,714.000000,714.000000,714.000000,714.000000,680.000000,678.000000,679.000000
mean,1056.854545,11.472727,23.263305,0.051821,2.388515,56.668067,1359.494398,0.041196,36.930678,24.939617
std,625.574471,6.928427,4.039645,0.221820,1.171270,19.661880,614.171870,0.020656,12.641503,10.075857
min,0.000000,0.000000,14.600000,0.000000,0.000000,16.000000,126.000000,0.003000,9.000000,7.000000
25%,507.000000,5.500000,20.300000,0.000000,1.500000,42.000000,807.250000,0.027000,28.000000,17.000000
50%,1046.000000,11.000000,22.900000,0.000000,2.300000,55.000000,1489.000000,0.039000,35.000000,24.000000
75%,1616.500000,17.000000,26.375000,0.000000,3.275000,73.750000,2000.000000,0.052250,45.000000,31.000000
max,2177.000000,23.000000,33.800000,1.000000,5.900000,100.000000,2000.000000,0.138000,94.000000,69.000000


In [38]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1459 non-null   int64  
 1   hour                    1459 non-null   int64  
 2   hour_bef_temperature    1457 non-null   float64
 3   hour_bef_precipitation  1457 non-null   float64
 4   hour_bef_windspeed      1450 non-null   float64
 5   hour_bef_humidity       1457 non-null   float64
 6   hour_bef_visibility     1457 non-null   float64
 7   hour_bef_ozone          1383 non-null   float64
 8   hour_bef_pm10           1369 non-null   float64
 9   hour_bef_pm2.5          1342 non-null   float64
 10  count                   1459 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 125.5 KB


In [39]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      715 non-null    int64  
 1   hour                    715 non-null    int64  
 2   hour_bef_temperature    714 non-null    float64
 3   hour_bef_precipitation  714 non-null    float64
 4   hour_bef_windspeed      714 non-null    float64
 5   hour_bef_humidity       714 non-null    float64
 6   hour_bef_visibility     714 non-null    float64
 7   hour_bef_ozone          680 non-null    float64
 8   hour_bef_pm10           678 non-null    float64
 9   hour_bef_pm2.5          679 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 56.0 KB


In [40]:
# 결측치 확인
print(train.isnull().sum())

id                          0
hour                        0
hour_bef_temperature        2
hour_bef_precipitation      2
hour_bef_windspeed          9
hour_bef_humidity           2
hour_bef_visibility         2
hour_bef_ozone             76
hour_bef_pm10              90
hour_bef_pm2.5            117
count                       0
dtype: int64


In [41]:
# 결측치 확인
print(test.isnull().sum())

id                         0
hour                       0
hour_bef_temperature       1
hour_bef_precipitation     1
hour_bef_windspeed         1
hour_bef_humidity          1
hour_bef_visibility        1
hour_bef_ozone            35
hour_bef_pm10             37
hour_bef_pm2.5            36
dtype: int64


In [42]:
# KNN imputation을 사용하여 결측치 채우기
# k=5로 설정하여 가장 유사한 5개의 데이터를 기반으로 결측치를 예측.
train_df_imputed = pd.DataFrame(KNN(k=5).fit_transform(train))

Imputing row 1/1459 with 0 missing, elapsed time: 0.209
Imputing row 101/1459 with 2 missing, elapsed time: 0.210
Imputing row 201/1459 with 0 missing, elapsed time: 0.210
Imputing row 301/1459 with 0 missing, elapsed time: 0.211
Imputing row 401/1459 with 0 missing, elapsed time: 0.211
Imputing row 501/1459 with 0 missing, elapsed time: 0.211
Imputing row 601/1459 with 0 missing, elapsed time: 0.212
Imputing row 701/1459 with 0 missing, elapsed time: 0.213
Imputing row 801/1459 with 0 missing, elapsed time: 0.213
Imputing row 901/1459 with 0 missing, elapsed time: 0.214
Imputing row 1001/1459 with 0 missing, elapsed time: 0.215
Imputing row 1101/1459 with 3 missing, elapsed time: 0.215
Imputing row 1201/1459 with 0 missing, elapsed time: 0.215
Imputing row 1301/1459 with 0 missing, elapsed time: 0.216
Imputing row 1401/1459 with 0 missing, elapsed time: 0.216


In [43]:
# KNN imputation을 사용하여 결측치 채우기
# k=5로 설정하여 가장 유사한 5개의 데이터를 기반으로 결측치를 예측.
test_df_imputed = pd.DataFrame(KNN(k=5).fit_transform(test))

Imputing row 1/715 with 0 missing, elapsed time: 0.058
Imputing row 101/715 with 0 missing, elapsed time: 0.059
Imputing row 201/715 with 0 missing, elapsed time: 0.059
Imputing row 301/715 with 0 missing, elapsed time: 0.059
Imputing row 401/715 with 0 missing, elapsed time: 0.059
Imputing row 501/715 with 0 missing, elapsed time: 0.060
Imputing row 601/715 with 0 missing, elapsed time: 0.060
Imputing row 701/715 with 0 missing, elapsed time: 0.060


In [53]:
train_df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1459 non-null   float64
 1   1       1459 non-null   float64
 2   2       1459 non-null   float64
 3   3       1459 non-null   float64
 4   4       1459 non-null   float64
 5   5       1459 non-null   float64
 6   6       1459 non-null   float64
 7   7       1459 non-null   float64
 8   8       1459 non-null   float64
 9   9       1459 non-null   float64
 10  10      1459 non-null   float64
dtypes: float64(11)
memory usage: 125.5 KB


In [54]:
test_df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       715 non-null    float64
 1   1       715 non-null    float64
 2   2       715 non-null    float64
 3   3       715 non-null    float64
 4   4       715 non-null    float64
 5   5       715 non-null    float64
 6   6       715 non-null    float64
 7   7       715 non-null    float64
 8   8       715 non-null    float64
 9   9       715 non-null    float64
dtypes: float64(10)
memory usage: 56.0 KB


In [45]:
# 대여량을 target으로 설정하고, 다른 변수들을 feature로 사용.
y = train_df_imputed.iloc[:, -1]
X = train_df_imputed.iloc[:, :-1]

In [46]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [47]:
# 랜덤포레스트 모델 학습
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [48]:
# test set 예측
y_pred = rf.predict(test_df_imputed)
y_pred

array([ 99.85, 215.5 ,  84.78,  30.71,  55.39, 119.15, 241.46, 272.3 ,
        35.97, 110.11, 306.41, 265.27, 120.41,  35.37, 215.29, 164.94,
        25.67, 176.11, 312.65, 169.63, 221.71,  68.35,  15.45, 144.91,
       142.31, 119.57,  29.72, 123.37, 116.35, 160.2 ,  72.14,  29.52,
        59.92, 138.52, 279.87,  32.64, 136.93, 111.76, 208.43,  76.82,
        50.12, 126.75, 169.67,  95.48, 308.79, 178.62,  77.47,  54.23,
        20.51,  86.93, 243.37,  88.72, 144.45,  60.57, 187.4 , 148.16,
        41.23, 178.34,  14.37,  18.06,  91.97,  90.13, 253.21, 290.18,
       142.26, 300.18,  16.12, 228.38, 160.52,  26.48, 104.32,  33.55,
       149.38,  15.18, 312.86, 230.  ,  31.91, 190.21, 255.94,  17.83,
       247.66, 142.5 ,  90.78,  84.95,  95.54, 324.72,  50.24, 160.87,
        94.09, 259.36, 261.78, 176.79,  58.62, 102.35,  36.34,  84.38,
        98.87,  27.43, 236.54, 146.  ,  18.04, 157.44,  41.29, 117.91,
        70.45,  51.1 , 102.16,  18.09, 178.02, 121.73, 181.43, 249.58,
      

In [49]:
submission = pd.read_csv('./data/submission.csv')
submission

,id,count
0,0,NaN
1,1,NaN
2,2,NaN
3,4,NaN
4,5,NaN
...,...,...
710,2148,NaN
711,2149,NaN
712,2165,NaN
713,2166,NaN


In [50]:
submission['count'] = y_pred

In [51]:
submission

,id,count
0,0,99.85
1,1,215.50
2,2,84.78
3,4,30.71
4,5,55.39
...,...,...
710,2148,47.89
711,2149,58.73
712,2165,115.52
713,2166,187.16


In [52]:
submission.to_csv('베이스라인.csv',index = False)